In [ ]:
!pip install accelerate -U
!pip install transformers soundfile datasets jiwer gdown pyctcdecode kenlm

In [23]:
!mkdir ./dataset
import gdown
def drive_download(idx, output):
    url = 'https://drive.google.com/uc?id=' + idx
    gdown.download(url, output, quiet=False)
drive_download("1ZBL3h6bHMmd8MIUNXqg72PucUkC9ZSWJ", "./dataset/train_data.zip")
# drive_download("1ZepptsTrVSjQEx-dpBBmQ2b7xYFLn_64", "./dataset/public_test.zip")
# drive_download("1K_07kix1OgBGO2FNPh-Lxqr1yLbtqFYt", "./dataset/train.jsonl")

mkdir: cannot create directory ‘./dataset’: File exists


Downloading...
From (uriginal): https://drive.google.com/uc?id=1ZepptsTrVSjQEx-dpBBmQ2b7xYFLn_64
From (redirected): https://drive.google.com/uc?id=1ZepptsTrVSjQEx-dpBBmQ2b7xYFLn_64&confirm=t&uuid=69efc103-117f-4b88-8754-1c91eb5131a7
To: /workspace/dataset/public_test.zip
100%|██████████| 131M/131M [00:01<00:00, 85.1MB/s] 


In [ ]:
!unzip ./dataset/public_test.zip -d ./dataset/test
!unzip ./dataset/train_data.zip -d ./dataset/train

In [2]:
import torch, json, utils, os
import numpy as np 
from functools import partial
from datasets import load_metric
from dataset import Wav2VecDataset
from torch.utils.data import DataLoader
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, TrainingArguments, Trainer

In [3]:
def custom_collate(processor, batch):
    processed_batch = [
        processor(i["input_values"], text=i["label"], sampling_rate=16000) for i in batch
    ]
    input_features = [{"input_values": i.input_values[0]} for i in processed_batch]
    input_features = processor.feature_extractor.pad(input_features, padding=True, return_tensors="pt")
    if batch[0]["label"] is None:
        return input_features
    label_features = [{"input_ids": i.labels} for i in processed_batch]
    label_features = processor.tokenizer.pad(
        label_features,
        padding=True,
        return_tensors="pt"
    )
    label_features = label_features["input_ids"].masked_fill(label_features.attention_mask.ne(1), -100)
    input_features["labels"] = label_features
    return input_features

In [4]:
def train_test_split(root_path, notation_file, test_size=0.3):
    notations = utils.load_annotation(notation_file)
    dataset = Wav2VecDataset(root_path, [i["file"] for i in notations], notations)
    N = len(dataset)
    print(f"Len dataset: {N}")
    train_size = int(N * (1-test_size))
    train_set, valid_set = torch.utils.data.random_split(dataset, [train_size, N-train_size])
    return train_set, valid_set

In [5]:
train_ds, valid_ds = train_test_split("./dataset/train/Train/", "./train.jsonl", test_size=0.3)
len(train_ds), len(valid_ds)

Len dataset: 7490


(5243, 2247)

In [6]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor
tokenizer = Wav2Vec2CTCTokenizer("./vn_base_vocab.json")
extractor = Wav2Vec2FeatureExtractor.from_pretrained("nguyenvulebinh/wav2vec2-large-vi-vlsp2020")

In [7]:
# nguyenvulebinh/wav2vec2-base-vietnamese-250h
processor = Wav2Vec2Processor(tokenizer=tokenizer, feature_extractor=extractor)
model = Wav2Vec2ForCTC.from_pretrained(
    "nguyenvulebinh/wav2vec2-large-vi-vlsp2020",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    # vocab_size=len(processor.tokenizer)
)
model.freeze_feature_encoder()
model.gradient_checkpointing_enable()
model.config.vocab_size = 111
# 768 - base model
model.lm_head = torch.nn.Linear(in_features=1024, out_features=111, bias=True)

Some weights of the model checkpoint at nguyenvulebinh/wav2vec2-large-vi-vlsp2020 were not used when initializing Wav2Vec2ForCTC: ['feature_transform.bn2.bias', 'feature_transform.bn2.running_var', 'feature_transform.bn3.running_var', 'feature_transform.bn1.running_var', 'feature_transform.bn1.weight', 'feature_transform.bn1.bias', 'feature_transform.bn3.running_mean', 'feature_transform.linear3.weight', 'feature_transform.linear3.bias', 'feature_transform.linear2.weight', 'feature_transform.bn3.weight', 'feature_transform.bn1.running_mean', 'feature_transform.bn2.weight', 'feature_transform.bn3.num_batches_tracked', 'feature_transform.bn3.bias', 'feature_transform.bn2.num_batches_tracked', 'feature_transform.linear2.bias', 'feature_transform.bn2.running_mean', 'feature_transform.linear1.bias', 'feature_transform.bn1.num_batches_tracked', 'feature_transform.linear1.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task 

In [8]:
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

/tmp/ipykernel_400/738329927.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [9]:
batch_size = 32
steps = len(train_ds) // batch_size
steps
# all_steps = 

163

In [12]:
training_args = TrainingArguments(
    output_dir="./wav2vec_large_st_1",
    save_total_limit=1,
    
    group_by_length=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="steps",
    fp16=True,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    warmup_steps=500//2,
    
    save_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    max_steps=steps*15,
    
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False
)

In [13]:
trainer = Trainer(
    model=model,
    data_collator=partial(custom_collate, processor),
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    tokenizer=processor.feature_extractor,
)

In [14]:
trainer.train()

Step,Training Loss,Validation Loss,Wer
163,13.892500,12.550158,1.000000
326,4.545400,4.260853,1.000000
489,3.446000,3.849141,1.000000
652,3.378500,3.476312,1.000000
815,3.274700,3.288993,0.999960
978,2.852800,1.872195,0.951879
1141,1.305100,0.699217,0.410770
1304,0.726400,0.456116,0.285528
1467,0.531000,0.358836,0.239422
1630,0.449100,0.309599,0.219865


TrainOutput(global_step=2445, training_loss=2.4054909610553263, metrics={'train_runtime': 2123.8124, 'train_samples_per_second': 36.839, 'train_steps_per_second': 1.151, 'total_flos': 1.009551511959126e+19, 'train_loss': 2.4054909610553263, 'epoch': 14.91})

In [ ]:
# TrainOutput(
#     global_step=2445, 
#     training_loss=2.4054909610553263, 
#     metrics={
#         'train_runtime': 2123.8124, 
#         'train_samples_per_second': 36.839, 
#         'train_steps_per_second': 1.151, 
#         'total_flos': 1.009551511959126e+19, 
#         'train_loss': 2.4054909610553263, 
#         'epoch': 14.91
#     }
# )

In [35]:
model.load_state_dict(torch.load("./wav2vec_large_st_1/checkpoint-2445/pytorch_model.bin"))

<All keys matched successfully>

In [36]:
notations = utils.load_annotation("./train.jsonl")
dataset = Wav2VecDataset("./dataset/train/Train/", [i["file"] for i in notations], notations)
len(dataset)

7490

In [38]:
batch_size = 16
steps = len(dataset) // batch_size
steps

468

In [39]:
training_args = TrainingArguments(
    output_dir="./wav2vec_large_st_2",
    save_total_limit=1,
    
    group_by_length=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    evaluation_strategy="steps",
    # fp16=True,
    gradient_checkpointing=True,
    learning_rate=1e-4,
    warmup_steps=1000,
    
    save_steps=steps,
    eval_steps=steps,
    logging_steps=steps,
    max_steps=steps*15,
    
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False
)
trainer = Trainer(
    model=model,
    data_collator=partial(custom_collate, processor),
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=processor.feature_extractor,
)
trainer.train()

Step,Training Loss,Validation Loss,Wer
468,0.324300,0.169837,0.143994
936,0.275700,0.138994,0.131735
1404,0.258000,0.115408,0.123338
1872,0.218900,0.097054,0.119487
2340,0.191800,0.091720,0.116001
2808,0.174600,0.087326,0.115766
3276,0.149700,0.073760,0.112256
3744,0.152400,0.071943,0.113881
4212,0.145600,0.063751,0.107922
4680,0.136100,0.060181,0.106603


TrainOutput(global_step=7020, training_loss=0.17307361211532202, metrics={'train_runtime': 4211.8219, 'train_samples_per_second': 26.668, 'train_steps_per_second': 1.667, 'total_flos': 1.4168328886976856e+19, 'train_loss': 0.17307361211532202, 'epoch': 14.97})

In [40]:
model.load_state_dict(torch.load("./wav2vec_large_st_2/checkpoint-5616/pytorch_model.bin"))

<All keys matched successfully>

In [41]:
import os
test_ds = Wav2VecDataset("./dataset/test/public_test", os.listdir("./dataset/test/public_test"))
len(test_ds) 

1299

In [42]:
def test_collate(processor, batch):
    processed_batch = [
        processor(i["input_values"], sampling_rate=i["sample_rate"]) for i in batch
    ]
    input_features = [{"input_values": i.input_values[0]} for i in processed_batch]
    input_features = processor.pad(input_features, padding=True, return_tensors="pt")
    input_features["id"] = [i["file"] for i in batch]
    return input_features

In [43]:
test_loader = DataLoader(test_ds, shuffle=False, batch_size=32, collate_fn=partial(test_collate, processor))
len(test_loader)

41

In [65]:
def wav2vec_inference(model, test_loader, processor, device=None):
    if device is None:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.eval() 
    model = model.to(device)
    pred_sentences = {}
    for idx, batch in enumerate(test_loader, 1):
        X_test =  batch["input_values"].to(device)
        X_test = X_test.half()
        file_test = batch["id"]
        with torch.set_grad_enabled(False):
            logits = model(input_values=X_test).logits
        logits = torch.argmax(logits, dim=-1)
        transcriptions = processor.batch_decode(logits, skip_special_tokens=True)
        for file_id, trans in zip(file_test, transcriptions):
            pred_sentences[file_id] = trans
        # pred_sentences += transcriptions
        print("\r", end="")
        print(f"\r {idx} / {len(test_loader)}", end = "" if idx != len(test_loader) else "\n")
    return pred_sentences

In [66]:
pred_sens = wav2vec_inference(model, test_loader, processor, torch.device("cuda:0"))

 41 / 41


In [67]:
with open("./w2v_lasted_test_sentences.json", "w", encoding="utf-8") as f:
    json.dump(pred_sens, f, ensure_ascii=False)
    f.close()